In [ ]:
!pip install langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 4.4 MB/s eta 0:00:00


In [ ]:
import json
import pickle
import pandas as pd
import numpy as np
from getpass import getpass
from tqdm import tqdm
from typing import List

from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [ ]:
GROQ_API_KEY = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [ ]:
llama3_70b_temp0 = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key=GROQ_API_KEY)

# Test
llama3_70b_temp0.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 17, 'total_tokens': 25, 'completion_time': 0.022857143, 'prompt_time': 0.00519721, 'queue_time': None, 'total_time': 0.028054353}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2f30b0b571', 'finish_reason': 'stop', 'logprobs': None}, id='run-cbe874a5-f949-41ba-ac92-a9d8f08b2aff-0')

In [ ]:
df = pd.read_csv("/content/dataset_concurso_rag.csv")
df.columns, len(df)

(Index(['pergunta', 'resposta', 'comentarios', 'numero_na_captura', 'id',
        'area', 'ano', 'banca', 'prova', 'duvidou_do_gabarito', 'enunciado',
        'assertivas', 'alternativas', 'classificacao',
        'artigos_nos_comentarios', 'artigos_completos', 'artigo_inventado',
        'topicos_para_busca', 'artigos_candidatos', 'frase_assunto_da_questao',
        'rag_art_inventado_full', 'rag_art_inventado_full_sem_art',
        'rag_assunto_full', 'rag_topicos_full', 'rag_artigos_candidatos',
        'rag_artigos_candidatos_grupos'],
       dtype='object'),
 37)

In [ ]:
def loop_fn(df, fn, kwarg=None):
    list_of_outputs = []
    for index, row in tqdm(df.iterrows(), total=len(df)):
        try:
            if kwarg:
                output = fn(row, kwarg)
            else:
                output = fn(row)
            print(output)
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            output = "Error"
        list_of_outputs.append(output)
    return list_of_outputs

In [ ]:
class BASELINE_JSON(BaseModel):
    analise: str = Field(description="Análise da questão, resposta e artigos de contexto.")
    resposta: str = Field(description="Resposta obtida pela análise dos trechos selecionados.")

output_parser = JsonOutputParser(pydantic_object=BASELINE_JSON)

In [ ]:
prompt = """
Você é professor de Direito e ávido conhecedor do Código Civil Brasileiro.
Use seus conhecimentos em leis e códigos analisar atentament os itens da questão.

Em seguida responda corretamente a questão, sinalizando na resposta somente a letra da alternativa.

questão: {questão}

Gere somente as informações dos campos do JSON, sem introduções ou formalismos.
Instruções de saída: {format_instructions}
""".strip()

prompt_template = PromptTemplate(
    input_variables=["questão"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [ ]:
def call_baseline(sample):
    output = chain.invoke(
        {"questão": sample['pergunta']})
    return output

# test
out = call_baseline(df.iloc[15])
out

{'analise': "A questão versa sobre o direito de família, especificamente sobre os alimentos. O Código Civil Brasileiro estabelece que os alimentos são devidos entre os parentes, inclusive entre pais e filhos. O art. 1.696 do CC/2002 estabelece que 'os alimentos são devidos pelos parentes em linha reta, em proporção às suas possibilidades'. Já o art. 1.697 do CC/2002 dispõe que 'o direito a alimentos é recíproco entre os parentes, estendendo-se aos avós, bisavós e demais ascendentes'.",
 'resposta': 'B'}

In [ ]:
list_of_outputs = loop_fn(df, call_baseline)

  3%|▎         | 1/37 [00:00<00:27,  1.32it/s]

{'analise': 'A questão envolve a situação de um herdeiro aparente e de boa-fé, que realizou benfeitorias no imóvel herdado e foi surpreendido com uma ação de nulidade de inventário cumulada com petição de herança. O Código Civil Brasileiro, em seu art. 1.815, estabelece que o herdeiro aparente e de boa-fé tem direito a todos os frutos percebidos e a indenização das benfeitorias necessárias, úteis e voluptuárias, podendo exercer o direito de retenção.', 'resposta': 'B'}


  5%|▌         | 2/37 [00:01<00:26,  1.32it/s]

{'analise': 'Análise da questão: A questão apresenta quatro afirmativas sobre a sucessão testamentária no Código Civil Brasileiro. Afirmativa I está correta, pois o art. 1.866 do Código Civil Brasileiro estabelece que o testamento público é o único permitido ao cego. Afirmativa II está incorreta, pois o testamento cerrado não precisa ser escrito mecanicamente. Afirmativa III está correta, pois o art. 1.879 do Código Civil Brasileiro estabelece que o testamento particular pode ser confirmado pelo juiz em circunstâncias excepcionais. Afirmativa IV está incorreta, pois o art. 1.876 do Código Civil Brasileiro estabelece que o testamento particular deve ser escrito em língua nacional.', 'resposta': 'B'}


  8%|▊         | 3/37 [00:02<00:26,  1.30it/s]

...

{'analise': "A legítima dos herdeiros necessários é a parte da herança que lhes é assegurada por lei, não podendo ser excluída pelo testador. De acordo com o art. 1.845 do Código Civil Brasileiro, 'a legítima dos herdeiros necessários é de 50% do patrimônio líquido do falecido'.", 'resposta': 'E'}


 97%|█████████▋| 36/37 [03:19<00:07,  7.03s/it]

{'analise': 'A questão envolve a prescrição da petição de herança em uma ação de investigação de paternidade post mortem. De acordo com o Código Civil Brasileiro, o prazo prescricional para a petição de herança é de 10 anos, contados da abertura da sucessão (art. 1.962 do CC). No caso, a abertura da sucessão ocorreu em 5/2/2010, e a petição inicial foi protocolada em 15/12/2022, portanto, dentro do prazo prescricional.', 'resposta': 'A'}


100%|██████████| 37/37 [03:24<00:00,  5.54s/it]

{'analise': 'De acordo com o art. 1.798 do Código Civil, o nascituro tem legitimidade para suceder, desde que nasça vivo. Portanto, Ana, como esposa de João, tem direito à sucessão e o nascituro, ainda não nascido, também terá direito à sucessão se nascer vivo.', 'resposta': 'C'}


In [ ]:
baseline_ds2 = pd.DataFrame(list_of_outputs)
baseline_ds2.to_csv("baseline_ds2.csv", index=False)

In [17]:
baseline_ds2['resposta']

0     B
1     B
2     B
3     D
4     A
5     C
6     C
7     C
8     E
9     B
10    C
11    C
12    C
13    E
14    E
15    B
16    C
17    B
18    D
19    A
20    D
21    A
22    C
23    C
24    C
25    E
26    E
27    E
28    A
29    E
30    B
31    C
32    A
33    C
34    E
35    A
36    C
Name: resposta, dtype: object

In [19]:
df['resposta']

0     C
1     B
2     E
3     C
4     C
5     E
6     E
7     E
8     D
9     B
10    A
11    B
12    C
13    E
14    E
15    B
16    C
17    A
18    D
19    A
20    B
21    A
22    E
23    C
24    C
25    E
26    C
27    E
28    B
29    E
30    C
31    D
32    A
33    A
34    C
35    E
36    C
Name: resposta, dtype: object

In [22]:
r_a = pd.DataFrame({
    'resposta_baseline': baseline_ds2['resposta'],
    'resposta': df['resposta'],
    'comparativo': baseline_ds2['resposta'] == df['resposta']})

r_a


,resposta_baseline,resposta,comparativo
0,B,C,False
1,B,B,True
2,B,E,False
3,D,C,False
4,A,C,False
5,C,E,False
6,C,E,False
7,C,E,False
8,E,D,False
9,B,B,True


In [23]:
r_a['comparativo'].value_counts()

comparativo
False    20
True     17
Name: count, dtype: int64

In [25]:
17/37

0.4594594594594595